### For developers wanting to extend the functionality of cyclops.query API for existing datasets or add new datasets, this in-depth guide can be a resource. The walkthrough explanations are using the MIMIC-IV dataset.

### This notebook dives into the details of how to use SQLAlchemy, which is the main python library used to develop the cyclops.query API. 

## SQLAlchemy

SQLAlchemy (https://www.sqlalchemy.org/) is a Python SQL toolkit which implements an Object Relational Mapper (ORM).

An ORM allows users to stay away writing queries as raw SQL strings, which become highly unmanageable as more complex queries are required.

In this tutorial, we'll introduce some basic functionalities provided by SQLAlchemy's ORM, as well as the functionality we've developed for simplifying and extending the use of this ORM.

Note: cyclops.query API is built for querying from a database, not on modifying it, e.g., inserting or deleting rows/tables.

## Query objects

There are a few objects representing queries/relations to understand:
 - `cyclops.query.utils.DBTable` (DBTable)
  - A class defined in this repository used by `cyclops.orm.Database` to organize tables as attributes
 - `sqlalchemy.sql.schema.Table` (Table)
  - The SQLAlchemy table object
  - The DBTable object has attribute `data` which accesses the corresponding Table object
 - `sqlalchemy.sql.selectable.Select` (Select)
  - The SQLAlchemy object returned by function `sqlalchemy.select`
  - Executing queries requires them to be Select objects
 - `sqlalchemy.sql.selectable.Subquery` (Subquery)
  - The SQLAlchemy object returned by method `.subquery()`, e.g., on a `sqlalchemy.select` object
  - This is necessary when wanting to chain queries together

## Query Interface

We wish to provide simple, and easy to use functions for querying a database such as MIMIC-IV, while leveraging the abstraction and flexibility provided by SQLAlchemy's ORM. 

In order to achieve this, the `cyclops.orm.Database` object which encapsulates the ORM, is hidden away from the user. Only the query and resulting data is made available using `cyclops.query.interface.QueryInterface`.

Hence, queries such patients, diagnoses, labs, etc. are encapsulated within this interface, and the user would execute the query using a simple `run()` method which belongs to the interface. The query is then executed and the resulting data as a `pandas.DataFrame` is stored in the interface's `data` attribute.

## API methods

Typically for health datasets collected from hospitals, such as MIMIC-IV, there are multiple tables where different sources of information are available. Usually the following sources are commonly available:

* Patients
* Diagnoses
* Events or Measurements
* Notes

Events can further be categorised into labs, vital measurements, interventions. A researcher or user is interested in developing risk-predictive models that predict outcomes of interest for a patient, which use patient statics and events/measurements as inputs.

Hence, as part of any dataset's query API, we can define functions that allow users to query the above information, and apply different filters to get subsets of interest. A researcher might be interested in patients belonging to a specific cohort depending on an outcome of interest, for example patients that have heart conditions or have had heart transplants. 

```python
    def patients():
        pass
    
    def diagnoses():
        pass
        

## Attribute access

Attributes, or columns, are accessed by name. How to access an column depends on the object.

In [74]:
# DBTable
col = db.mimic_hosp.d_icd_diagnoses.icd_code
type(col)

sqlalchemy.sql.schema.Column

<IPython.core.display.Javascript object>

There is no convenient attribute access for a `sqlalchemy.sql.schema.Table` object. This was largely the motivation for creating the `cyclops.query_utils.DBTable` object.

There is no direct attribute access for a `sqlalchemy.sql.selectable.Select` object. Instead, we must first convert it to a subquery.

In [75]:
# Subquery
subquery = select(db.mimic_hosp.d_icd_diagnoses.data).subquery()
type(subquery.c.icd_code)

sqlalchemy.sql.schema.Column

<IPython.core.display.Javascript object>

Note that difference in attribute access between a DBTable and a subquery, where in a subquery, we must use access `.c` before accessing column.

# Helper functions for querying

SQLAlchemy does not have implementations for many desired functionalities.

Here, we introduce `cyclops.query.utils` (imported as `query_utils`), which provides general-purpose functions we'll use throughout the remainder of the notebook to make building effective queries quick and simple. These helper functions can be used to add/improve the higher level API functions in the dataset API modules such as `cyclops.query.mimic`.

 - Handling conditions: `equals`, `in_`, `has_string_format`, `has_substring`, `starts_with`, `ends_with`
 - Attribute handling: `get_attributes`, `rename_attributes`, `reorder_attributes`, `drop_attributes`, `apply_to_attributes`
 - It automatically handles conversions between different query type objects, so users need not worry about what to pass in

## Conditions

Row conditions are represented by the `sqlalchemy.sql.elements.BinaryExpression` object.

In [76]:
cond = db.mimic_hosp.diagnoses_icd.icd_code == "0011"
type(cond)

sqlalchemy.sql.elements.BinaryExpression

<IPython.core.display.Javascript object>

We can use method `where` to use a binary expression object to filter rows of a `sqlalchemy.sql.selectable.Select` object:

In [77]:
cond = db.mimic_hosp.d_icd_diagnoses.icd_code == "0010"
query = select(db.mimic_hosp.d_icd_diagnoses.data).where(cond)
db.run_query(query)

2022-03-31 09:44:58,025 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:58,026 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.016161 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.016161 s


,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae


<IPython.core.display.Javascript object>

Method `filter` may be used equivalently:

In [78]:
cond = db.mimic_hosp.d_icd_diagnoses.icd_code == "0010"
query = select(db.mimic_hosp.d_icd_diagnoses.data).filter(cond)
db.run_query(query)

2022-03-31 09:44:58,059 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:58,060 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.015436 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.015436 s


,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae


<IPython.core.display.Javascript object>

More complex conditions have been handled in `cyclops.query_utils`, available for convenience.

Consider the substring condition:

In [79]:
cond = q_utils.substring_cond(db.mimic_hosp.d_icd_diagnoses.long_title, "Heart-")
query = select(db.mimic_hosp.d_icd_diagnoses.data).where(cond)
db.run_query(query)

2022-03-31 09:44:58,170 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:58,172 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.097989 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.097989 s


,icd_code,icd_version,long_title
0,T863,10,Complications of heart-lung transplant
1,T8630,10,Unspecified complication of heart-lung transplant
2,T8631,10,Heart-lung transplant rejection
3,T8632,10,Heart-lung transplant failure
4,T8633,10,Heart-lung transplant infection
5,T8639,10,Other complications of heart-lung transplant
6,Z48280,10,Encounter for aftercare following heart-lung t...
7,Z954,10,Presence of other heart-valve replacement


<IPython.core.display.Javascript object>

Notice that we received results with "heart-" despite specifying "Heart-". This is because the function automatically assumed we wanted to ignore case. Different pre-processing assumptions are made for condition functions and can be explored in `cyclops.query_utils`.

These pre-processing assumptions are entirely optional. If case matters, we could simply specify this:

In [80]:
cond = q_utils.substring_cond(
    db.mimic_hosp.d_icd_diagnoses.long_title, "Heart-", lower=False
)
query = select(db.mimic_hosp.d_icd_diagnoses.data).where(cond)
db.run_query(query)

2022-03-31 09:44:58,229 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:58,230 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.034915 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.034915 s


,icd_code,icd_version,long_title
0,T8631,10,Heart-lung transplant rejection
1,T8632,10,Heart-lung transplant failure
2,T8633,10,Heart-lung transplant infection


<IPython.core.display.Javascript object>

Assumptions may be made regarding case, trimming of whitespace, and type conversions.

## Joins

We can perform joins in SQLAlchemy.

For example, say we want to get a table back with patient diagnoses (found in `db.mimic_hosp.diagnoses_icd`), but include the titles of the diagnoses (found in `db.mimic_hosp.d_icd_diagnoses`).

When creating complex queries, many joins consist of at least one, if not two, subqueries. So, we'll demonstrate an inner join using a subquery:

In [81]:
query = select(db.mimic_hosp.diagnoses_icd.data, subquery).join(
    subquery, db.mimic_hosp.diagnoses_icd.icd_code == subquery.c.icd_code
)
db.run_query(query, limit=3)

2022-03-31 09:44:58,558 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:58,560 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.314942 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.314942 s


,subject_id,hadm_id,seq_num,icd_code,icd_version,icd_code_1,icd_version_1,long_title
0,15998114,22987966,2,V6284,9,V6284,9,Suicidal ideation
1,19754677,27138064,8,71590,9,71590,9,"Osteoarthrosis, unspecified whether generalize..."
2,19754677,27138064,2,42833,9,42833,9,Acute on chronic diastolic heart failure


<IPython.core.display.Javascript object>

A cross product can be done by simply not using the `join` method:

In [82]:
subquery = select(db.mimic_hosp.d_icd_diagnoses.data).subquery()
query = select(db.mimic_hosp.diagnoses_icd.data, subquery)
db.run_query(query, limit=3)

2022-03-31 09:44:58,577 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:58,578 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.003934 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.003934 s


,subject_id,hadm_id,seq_num,icd_code,icd_version,icd_code_1,icd_version_1,long_title
0,12926949,21005278,4,79311,9,0010,9,Cholera due to vibrio cholerae
1,17995056,24718820,3,V6284,9,0010,9,Cholera due to vibrio cholerae
2,17995056,24718820,5,25000,9,0010,9,Cholera due to vibrio cholerae


<IPython.core.display.Javascript object>

It is smart to test queries with joins using a small `limit` in `db.run_query`, since cross products can create unreasonably large tables.

Notice that in the inner join we have duplicated, unwanted columns `icd_code_1`, `icd_version_1`. This is because we selected the entire `db.mimic_hosp.d_icd_diagnoses` table despite only wanting the `long_title` column.

We can fix this by selecting this column specifically:

In [83]:
subquery = select(db.mimic_hosp.d_icd_diagnoses.data).subquery()

running_query = select(db.mimic_hosp.diagnoses_icd.data, subquery.c.long_title).join(
    subquery, db.mimic_hosp.diagnoses_icd.icd_code == subquery.c.icd_code
)
db.run_query(running_query, limit=3)

2022-03-31 09:44:58,901 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:58,903 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.310452 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.310452 s


,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,17190208,26495862,38,Y838,10,Other surgical procedures as the cause of abno...
1,19754677,27138064,8,71590,9,"Osteoarthrosis, unspecified whether generalize..."
2,19754677,27138064,2,42833,9,Acute on chronic diastolic heart failure


<IPython.core.display.Javascript object>

We can select any combination of tables and columns in this manner.

## Group by, Order by

We can perform use the `group_by` method to group by different columns:

In [84]:
query = select(
    db.mimic_hosp.diagnoses_icd.icd_code,
    func.count(db.mimic_hosp.diagnoses_icd.icd_code),
).group_by(db.mimic_hosp.diagnoses_icd.icd_code)
db.run_query(query, limit=3)

2022-03-31 09:44:59,298 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:59,299 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.383287 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.383287 s


,icd_code,count_1
0,0020,8
1,0029,1
2,0030,34


<IPython.core.display.Javascript object>

The typically restrictions apply that anything selected must be in an aggregate function or included in the group by.

We can also take the previous query and order the rows in ascending or descending order:

In [85]:
# Descending
query = query.order_by(db.mimic_hosp.diagnoses_icd.icd_code.desc())
db.run_query(query, limit=3)

2022-03-31 09:44:59,687 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:44:59,689 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.379678 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.379678 s


,icd_code,count_1
0,Z9989,8
1,Z9981,3050
2,Z993,1841


<IPython.core.display.Javascript object>

In [86]:
# Back to ascending
subquery = query.subquery()
query = select(subquery).order_by(subquery.c.icd_code)
db.run_query(query, limit=3)

2022-03-31 09:45:00,122 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:00,124 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.414600 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.414600 s


,icd_code,count_1
0,0020,8
1,0029,1
2,0030,34


<IPython.core.display.Javascript object>

## Helper functions

Let's re-order the attributes in previously defined query, perhaps where we want `icd_code` and its corresponding title, `long_title`, next to one another:

In [87]:
running_query = q_utils.reorder_attributes(
    running_query,
    ["subject_id", "hadm_id", "seq_num", "icd_code", "long_title", "icd_version"],
)

db.run_query(running_query, limit=2)

2022-03-31 09:45:02,336 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:02,338 INFO cyclops.utils.profile - Finished executing function wrapper_func in 2.191950 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 2.191950 s


,subject_id,hadm_id,seq_num,icd_code,long_title,icd_version
0,10158684,22072764,1,0020,Typhoid fever,9
1,11816842,25448380,1,0020,Typhoid fever,9


<IPython.core.display.Javascript object>

Note that all existing attributes should be passed into this function, otherwise we'll receive an error.

We can also drop any attributes we might want removed:

In [88]:
query_drop = q_utils.drop_attributes(running_query, "seq_num")
db.run_query(query_drop, limit=2)

2022-03-31 09:45:05,155 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:05,157 INFO cyclops.utils.profile - Finished executing function wrapper_func in 2.795270 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 2.795270 s


,subject_id,hadm_id,icd_code,long_title,icd_version
0,10158684,22072764,0020,Typhoid fever,9
1,11816842,25448380,0020,Typhoid fever,9


<IPython.core.display.Javascript object>

Like many of the `q_utils` functions which accept attributes, we may pass in a single attribute, or multiple:

In [89]:
query_drop = q_utils.drop_attributes(running_query, ["seq_num", "long_title"])
db.run_query(query_drop, limit=2)

2022-03-31 09:45:05,239 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:05,241 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.060159 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.060159 s


,subject_id,hadm_id,icd_code,icd_version
0,12926949,21005278,79311,9
1,17995056,24718820,V6284,9


<IPython.core.display.Javascript object>

We can rename attributes:

In [90]:
query_rename = q_utils.rename_attributes(running_query, {"long_title": "icd_title"})
db.run_query(query_rename, limit=2)

2022-03-31 09:45:07,433 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,434 INFO cyclops.utils.profile - Finished executing function wrapper_func in 2.170368 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 2.170368 s


,subject_id,hadm_id,seq_num,icd_code,icd_title,icd_version
0,10158684,22072764,1,0020,Typhoid fever,9
1,11816842,25448380,1,0020,Typhoid fever,9


<IPython.core.display.Javascript object>

### Applying functions

We need to be careful. Consider the diagnosis ICD code values:

In [91]:
query = db.mimic_hosp.d_icd_diagnoses
df = db.run_query(query, limit=2)
df

2022-03-31 09:45:07,461 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,462 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.004560 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.004560 s


,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor


<IPython.core.display.Javascript object>

In [92]:
df["icd_code"].values

array(['0010   ', '0011   '], dtype=object)

<IPython.core.display.Javascript object>

There is lots of trailing whitespace which we may not have expected.

There are built-in function in SQLAlchemy such as sqlalchemy.func.trim which can remove leading/trailing whitespace, however it is difficult to apply these function in-place. We have created functionality for this.

Here, we use `q_utils.trim_attributes` to trim leading/trailing whitespace from an attribute:

In [93]:
query = q_utils.trim_attributes(db.mimic_hosp.d_icd_diagnoses, "icd_code")
df = db.run_query(query, limit=2)
df["icd_code"].values

2022-03-31 09:45:07,486 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,487 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.004210 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.004210 s


array(['0010', '0011'], dtype=object)

<IPython.core.display.Javascript object>

It is smart to perform sanity checks:

In [94]:
t = db.mimic_hosp.d_icd_diagnoses
df = db.run_query(t, limit=5)
icd_codes_untrimmed = df["icd_code"].values.astype("str")
np.char.strip(icd_codes_untrimmed) == icd_codes_untrimmed  # Trim using NumPy

2022-03-31 09:45:07,498 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,499 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.002870 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.002870 s


array([False, False, False, False, False])

<IPython.core.display.Javascript object>

In [95]:
query = q_utils.trim_attributes(t, "icd_code")
df = db.run_query(query, limit=5)
icd_codes_trimmed = df["icd_code"].values.astype("str")
np.char.strip(icd_codes_trimmed) == icd_codes_trimmed  # Trim using NumPy

2022-03-31 09:45:07,512 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,513 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.003301 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.003301 s


array([ True,  True,  True,  True,  True])

<IPython.core.display.Javascript object>

Like many of the functions which accept attributes, we may pass in a single attribute, or multiple:

In [96]:
query = q_utils.trim_attributes(
    db.mimic_hosp.d_icd_diagnoses, ["icd_code", "icd_version"]
)
df = db.run_query(query, limit=2)
df[["icd_code", "icd_version"]].values

2022-03-31 09:45:07,526 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,527 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.003709 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.003709 s


array([['0010', '9'],
       ['0011', '9']], dtype=object)

<IPython.core.display.Javascript object>

Note that certain functions will assume certain datatype conversions. For example, when trimming, the column would be converted to a string in order for this operation to be defined.

The ICD version was originally not a string:

In [97]:
query = db.mimic_hosp.d_icd_diagnoses
df = db.run_query(query, limit=2)
df["icd_version"].values

2022-03-31 09:45:07,539 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,540 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.002712 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.002712 s


array([9, 9])

<IPython.core.display.Javascript object>

String conversions are made with `string_format_cond`, `substring_cond`, `startswith_cond`, `endswith_cond`. For example,

In [98]:
query = select(db.mimic_hosp.diagnoses_icd.data).where(
    q_utils.startswith_cond(db.mimic_hosp.diagnoses_icd.icd_version, 1)
)
db.run_query(query, limit=3)

2022-03-31 09:45:07,551 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,552 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.003051 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.003051 s


,subject_id,hadm_id,seq_num,icd_code,icd_version
0,17190208,26495862,29,Y92239,10
1,17190208,26495862,38,Y838,10
2,17190208,26495862,20,K91840,10


<IPython.core.display.Javascript object>

There is also support for creating custom in-place functions using `q_utils.apply_to_attributes`.

Here we will use `sqlalchemy.func.lower` to convert the strings in a column to lowercase:

In [99]:
query = q_utils.apply_to_attributes(
    db.mimic_hosp.d_icd_diagnoses, "long_title", func.lower
)
db.run_query(query, limit=2)

2022-03-31 09:45:07,567 INFO cyclops.orm     - Query returned successfully!
INFO:cyclops.orm:Query returned successfully!
2022-03-31 09:45:07,568 INFO cyclops.utils.profile - Finished executing function wrapper_func in 0.004373 s
INFO:cyclops.utils.profile:Finished executing function wrapper_func in 0.004373 s


,icd_code,icd_version,long_title
0,0010,9,cholera due to vibrio cholerae
1,0011,9,cholera due to vibrio cholerae el tor


<IPython.core.display.Javascript object>

In particular, `q_utils.apply_to_attributes` accepts any function which takes a Column object as its only argument and similarly returns a Column object.